<a href="https://colab.research.google.com/github/PushkarGarg7/RAG_langchain/blob/main/Summarizer_OPENAI_BIGDATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


In [18]:
import openai

In [16]:
import pandas as pd

df=pd.read_csv("logistics_service_responses 2.csv")
df2=df.head(500000)
text_responses=df2["Response"].dropna().tolist()
# Define the path to the output text file
output_file_path = 'text_responses2.txt'

# Write the text responses to the text file
with open(output_file_path, 'w') as file:
    for response in text_responses:
        file.write(response + '\n')

print(f'Text responses have been written to {output_file_path}')
with open('text_responses2.txt', 'r') as file:
    feedback = file.read()

# Eliminate newline tokens
feedback = feedback.replace('\n', ' ')
# print(feedback)
with open('processed_feedback2.txt', 'w') as file:
    file.write(feedback)

Text responses have been written to text_responses2.txt


In [ ]:
from openai import OpenAI
client = OpenAI(api_key="APIKEY")   #set environment instead

AttributeError: 'DataFrame' object has no attribute 'random'

In [ ]:
# my_assistants = client.beta.assistants.list(
#     order="desc",
#     limit="20",
# )
# print(my_assistants.data)

[]


In [52]:
file_path = 'text_responses3.txt'
with open(file_path, 'rb') as f:
    file_response = client.files.create(
        file=f,
        purpose='assistants',
    )
    print(file_response)

file_id = file_response.id

FileObject(id='file-PqYWF09Lqy2qLOpQt7nVO3yw', bytes=33439946, created_at=1718278776, filename='text_responses3.txt', object='file', purpose='assistants', status='processed', status_details=None)


In [25]:
prompt = """
    These responses are customer survey responses by customers for a particular survey. The agent has to extract the info from the file as follows:
    1. General summary of 5-6 lines, like most users think this that etc.
    2. Pain points: What were the most frequent disliked things by users.
    3. Suggestions: What can you imporve.
    """

In [ ]:

for i in assistants:
  client.beta.assistants.delete(i["id"])


In [30]:
# assistant = client.beta.assistants.create(
#   name="Sur",
#   instructions="You have conducted a survey for your service/product and you have collected survey responses for it, your goal is to analyse those responses and give a conscise summary with key points.",
#   tools=[{"type": "code_interpreter"}],
#   model="gpt-3.5-turbo",
#   tool_resources={
#     "code_interpreter": {
#       "file_ids": [file.id]
#     }
#   }
# )
assistant_response = client.beta.assistants.create(
    name="Survey Summarizer",
    description="Analyze customer survey responses and provide insights",
    model="gpt-3.5-turbo",
    instructions="You have conducted a survey for your service/product and you have collected survey responses for it. Your goal is to analyze those responses and give a concise summary with key points.",
    tools=[
        {
            "type": "code_interpreter"
        }
    ]
)
assistant_id = assistant_response.id

In [31]:
assistant_response

Assistant(id='asst_9YQ8eYduLTsuqdAg1JASiO4Z', created_at=1718276838, description='Analyze customer survey responses and provide insights', instructions='You have conducted a survey for your service/product and you have collected survey responses for it. Your goal is to analyze those responses and give a concise summary with key points.', metadata={}, model='gpt-3.5-turbo', name='Survey Summarizer', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0)

In [53]:
prompt="""
Give a concise summary from the given customer responses talking about what the customers liked, disliked and what can the organisation do to imporve.
Return Format :
Summary : (5-6 lines giving general overview)
What customers liked : in 2-3 lines
What customers disliked : in 2-3 lines
What can you imporve : Suggestions for improvement from your side based of responses.
Don't return anything else.
"""

In [54]:
# thread = client.beta.threads.create(
#   messages=[
#     {
#       "role": "user",
#       "content": prompt,
#       "attachments": [
#         {
#           "file_id": file.id,
#           "tools": [{"type": "code_interpreter"}]
#         }
#       ]
#     }
#   ]
# )
message_thread = client.beta.threads.create(
    messages=[
    {
      "role": "user",
      "content": prompt,
      "attachments": [
        {
          "file_id": file_id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]

)
message_thread
thread_id=message_thread.id

In [55]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread_id,
  assistant_id=assistant_id,
  model="gpt-4o"

)

In [56]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread_id
  )
  print(messages)
else:
  print(run.status)
print("    ")
print("gap1")
print("    ")

messages=client.beta.threads.messages.list(
    thread_id=thread_id
)
print(messages)

SyncCursorPage[Message](data=[Message(id='msg_OLoMGOJUmeNaFHMxJWf2cWjV', assistant_id='asst_9YQ8eYduLTsuqdAg1JASiO4Z', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Summary: Customers generally found the service ranging from satisfactory to fantastic, with varying opinions on specific aspects such as tracking updates and delivery times. There is a mix of delight and dissatisfaction, indicating room for improvement to elevate customer experiences universally.\n\nWhat customers liked: Many customers praised the service, with words like "fantastic," "impressive," and "exceeded my expectations." Some highlighted it as the best logistics service they have used.\n\nWhat customers disliked: A notable number of customers mentioned issues related to delays in delivery and inadequate tracking updates. Some also expressed frustration over having to follow up multiple times.\n\nWhat can you improve: Enhance the tracking system to provide more accurat